In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
import math

In [3]:
# Opening JSON file
f = open('twitter_aapl_holdout.json',)
  
# returns JSON object as 
# a dictionary
aapl = json.load(f)

f.close()

In [4]:
flataapl  = [val for sublist in aapl for val in sublist]

In [5]:
print(flataapl[0])

{'referenced_tweets': [{'type': 'retweeted', 'id': '1414164954375094276'}], 'id': '1414175172081373189', 'text': 'RT @begeek: Même Apple a eu du mal à croire en l’impressionnante gestion énergétique de sa puce M1 https://t.co/OxH8Qn9QNZ #Tech #Apple htt…', 'lang': 'fr', 'created_at': '2021-07-11T10:50:09.000Z'}


In [6]:
df = pd.DataFrame(flataapl)
df

,referenced_tweets,id,text,lang,created_at,withheld
0,"[{'type': 'retweeted', 'id': '1414164954375094...",1414175172081373189,RT @begeek: Même Apple a eu du mal à croire en...,fr,2021-07-11T10:50:09.000Z,NaN
1,"[{'type': 'retweeted', 'id': '1414173981448495...",1414175103844188162,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:53.000Z,NaN
2,"[{'type': 'retweeted', 'id': '1414071661767372...",1414175068737867777,RT @AirChatter: -@B1 makeover and #BullishOnEO...,en,2021-07-11T10:49:45.000Z,NaN
3,NaN,1414175039759405062,"Hey startups, what are you doing with your end...",en,2021-07-11T10:49:38.000Z,NaN
4,NaN,1414174980867117059,俺が、顔出しして、しゃべるの得意なの知ってるだろ？\nhttps://t.co/yrTAJh...,ja,2021-07-11T10:49:24.000Z,NaN
...,...,...,...,...,...,...
448113,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12T13:00:30.000Z,NaN
448114,"[{'type': 'retweeted', 'id': '1392460622835720...",1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19.000Z,NaN
448115,"[{'type': 'retweeted', 'id': '1392434464500764...",1392464604186349570,RT @rokkua6: ✨✨５月の抽選大企画中✨✨\n\n今回も...？\n多くの参加者お...,ja,2021-05-12T13:00:07.000Z,NaN
448116,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12T13:00:02.000Z,NaN


In [7]:
y = df['withheld'].dropna()
y

64853     {'copyright': False, 'country_codes': ['IN']}
79768     {'copyright': False, 'country_codes': ['IN']}
80017     {'copyright': False, 'country_codes': ['IN']}
104260    {'copyright': False, 'country_codes': ['TR']}
104532    {'copyright': False, 'country_codes': ['TR']}
                              ...                      
360098    {'copyright': False, 'country_codes': ['IN']}
360134    {'copyright': False, 'country_codes': ['IN']}
360200    {'copyright': False, 'country_codes': ['IN']}
360203    {'copyright': False, 'country_codes': ['IN']}
438169    {'copyright': False, 'country_codes': ['RU']}
Name: withheld, Length: 105, dtype: object

In [8]:
df1 = df.drop('withheld',axis=1)
df1['created_at'] = df1['created_at'].map(lambda x: str(x)[:-5])
df1

,referenced_tweets,id,text,lang,created_at
0,"[{'type': 'retweeted', 'id': '1414164954375094...",1414175172081373189,RT @begeek: Même Apple a eu du mal à croire en...,fr,2021-07-11T10:50:09
1,"[{'type': 'retweeted', 'id': '1414173981448495...",1414175103844188162,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:53
2,"[{'type': 'retweeted', 'id': '1414071661767372...",1414175068737867777,RT @AirChatter: -@B1 makeover and #BullishOnEO...,en,2021-07-11T10:49:45
3,NaN,1414175039759405062,"Hey startups, what are you doing with your end...",en,2021-07-11T10:49:38
4,NaN,1414174980867117059,俺が、顔出しして、しゃべるの得意なの知ってるだろ？\nhttps://t.co/yrTAJh...,ja,2021-07-11T10:49:24
...,...,...,...,...,...
448113,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12T13:00:30
448114,"[{'type': 'retweeted', 'id': '1392460622835720...",1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19
448115,"[{'type': 'retweeted', 'id': '1392434464500764...",1392464604186349570,RT @rokkua6: ✨✨５月の抽選大企画中✨✨\n\n今回も...？\n多くの参加者お...,ja,2021-05-12T13:00:07
448116,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12T13:00:02


In [11]:
dfsmall = df1.iloc[0:10]
dfsmall
df2 = df1

In [12]:
# d = df1.loc[df1['text'] == str("It's no secret that Apple products are pricey")]
# g = df1.loc[df1['id'] == str(1259825816600444929)]
# print(g)
# g.iloc[0,0]
df1['id'].isnull().values.any()

False

In [13]:
dfcut = df2
dfcut = dfcut.dropna(subset=['referenced_tweets'])
dfcut['referenced_tweets'] = dfcut['referenced_tweets'].map(lambda x: x[0]['id'])
dfcut

<ipython-input-13-400ff7227f34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfcut['referenced_tweets'] = dfcut['referenced_tweets'].map(lambda x: x[0]['id'])


,referenced_tweets,id,text,lang,created_at
0,1414164954375094276,1414175172081373189,RT @begeek: Même Apple a eu du mal à croire en...,fr,2021-07-11T10:50:09
1,1414173981448495104,1414175103844188162,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:53
2,1414071661767372805,1414175068737867777,RT @AirChatter: -@B1 makeover and #BullishOnEO...,en,2021-07-11T10:49:45
6,1414173981448495104,1414174938345480192,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:14
9,1414143908469043204,1414174716013674500,RT @seriseri_gadget: 💡iPadを使いこなそう💡\n\niPadと一緒に...,ja,2021-07-11T10:48:21
...,...,...,...,...,...
448108,1392167552009359365,1392464834923352082,RT @domhofdesign: I recreated the macOS magnif...,en,2021-05-12T13:01:02
448109,1392460622835720195,1392464789927075840,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:51
448110,1390606669449629696,1392464771883048968,RT @rokkua6: ＼＼５月の抽選大企画／／\n\n多くの参加者お待ちしてます!!\n...,ja,2021-05-12T13:00:47
448114,1392460622835720195,1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19


In [14]:
# print(isinstance(df1.head(10)['referenced_tweets'].values, float))
# df2['rt'] = ""
# df2.loc[isinstance(df2['referenced_tweets'].values, float) == False, 'rt'] = df2['referenced_tweets']

def text_iter4(ref_tweets,twid):
    dfcut.loc[twid == ref_tweets,'fulltext'] = df2.loc[twid == ref_tweets, 'text']
#     dfcut['fulltext'] = 
#     df2.loc[df2['id'] = x]
#     df2['fulltext'] = 
text_iter4(dfcut['referenced_tweets'].values,df2['id'].values)
dfcut
# for index, row in df1.head(10).iterrows():

#     print(np.array(df2['referenced_tweets'][index])[0]['id'])

<ipython-input-14-e0f892d7eb3e>:6: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  dfcut.loc[twid == ref_tweets,'fulltext'] = df2.loc[twid == ref_tweets, 'text']


KeyError: False

In [15]:
dftemp = pd.merge(left=dfcut,right=df2,how='left',left_on='referenced_tweets',right_on='id')
dftemp

,referenced_tweets_x,id_x,text_x,lang_x,created_at_x,referenced_tweets_y,id_y,text_y,lang_y,created_at_y
0,1414164954375094276,1414175172081373189,RT @begeek: Même Apple a eu du mal à croire en...,fr,2021-07-11T10:50:09,NaN,1414164954375094276,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11T10:09:33
1,1414173981448495104,1414175103844188162,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:53,NaN,1414173981448495104,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:45:25
2,1414071661767372805,1414175068737867777,RT @AirChatter: -@B1 makeover and #BullishOnEO...,en,2021-07-11T10:49:45,NaN,1414071661767372805,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11T03:58:51
3,1414173981448495104,1414174938345480192,RT @scpwords: Time for ARCADEFEST! HTML5 games...,en,2021-07-11T10:49:14,NaN,1414173981448495104,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:45:25
4,1414143908469043204,1414174716013674500,RT @seriseri_gadget: 💡iPadを使いこなそう💡\n\niPadと一緒に...,ja,2021-07-11T10:48:21,NaN,1414143908469043204,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11T08:45:55
...,...,...,...,...,...,...,...,...,...,...
251131,1392167552009359365,1392464834923352082,RT @domhofdesign: I recreated the macOS magnif...,en,2021-05-12T13:01:02,NaN,NaN,NaN,NaN,NaN
251132,1392460622835720195,1392464789927075840,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:51,NaN,NaN,NaN,NaN,NaN
251133,1390606669449629696,1392464771883048968,RT @rokkua6: ＼＼５月の抽選大企画／／\n\n多くの参加者お待ちしてます!!\n...,ja,2021-05-12T13:00:47,NaN,NaN,NaN,NaN,NaN
251134,1392460622835720195,1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19,NaN,NaN,NaN,NaN,NaN


In [16]:
newtext = np.array([])
def textthing(x,y):
    for index in range(len(dftemp)):
        if y[index] == y[index]:
            x[index] = y[index]
    print(x.size)
#     dftemp['newtext'] = x
textthing(dftemp['text_x'].values,dftemp['text_y'].values)

dftemp

251136


,referenced_tweets_x,id_x,text_x,lang_x,created_at_x,referenced_tweets_y,id_y,text_y,lang_y,created_at_y
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11T10:50:09,NaN,1414164954375094276,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11T10:09:33
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:53,NaN,1414173981448495104,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:45:25
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11T10:49:45,NaN,1414071661767372805,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11T03:58:51
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:14,NaN,1414173981448495104,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:45:25
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11T10:48:21,NaN,1414143908469043204,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11T08:45:55
...,...,...,...,...,...,...,...,...,...,...
251131,1392167552009359365,1392464834923352082,RT @domhofdesign: I recreated the macOS magnif...,en,2021-05-12T13:01:02,NaN,NaN,NaN,NaN,NaN
251132,1392460622835720195,1392464789927075840,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:51,NaN,NaN,NaN,NaN,NaN
251133,1390606669449629696,1392464771883048968,RT @rokkua6: ＼＼５月の抽選大企画／／\n\n多くの参加者お待ちしてます!!\n...,ja,2021-05-12T13:00:47,NaN,NaN,NaN,NaN,NaN
251134,1392460622835720195,1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19,NaN,NaN,NaN,NaN,NaN


In [41]:
dftemp2 = dftemp.iloc[: , :-5]
dftemp2.columns=['referenced_tweets','id','text','lang','created_at']
dftemp2

,referenced_tweets,id,text,lang,created_at
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11T10:50:09
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:53
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11T10:49:45
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:14
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11T10:48:21
...,...,...,...,...,...
251131,1392167552009359365,1392464834923352082,RT @domhofdesign: I recreated the macOS magnif...,en,2021-05-12T13:01:02
251132,1392460622835720195,1392464789927075840,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:51
251133,1390606669449629696,1392464771883048968,RT @rokkua6: ＼＼５月の抽選大企画／／\n\n多くの参加者お待ちしてます!!\n...,ja,2021-05-12T13:00:47
251134,1392460622835720195,1392464656912900100,"RT @JonyIveParody: “Hey Siri, how does iCloud ...",en,2021-05-12T13:00:19


In [61]:
dfother = df2
dfother = dfother[dfother['referenced_tweets'].isna()]
dfother.reset_index()
dffinal = pd.concat([dftemp2, dfother], ignore_index=True)
dffinal

,referenced_tweets,id,text,lang,created_at
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11T10:50:09
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:53
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11T10:49:45
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11T10:49:14
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11T10:48:21
...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12T13:00:44
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12T13:00:41
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12T13:00:30
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12T13:00:02


In [45]:
# for index, row in df1.head(599999).iterrows():
# #     a = row['referenced_tweets']
#     if isinstance(df1['referenced_tweets'][index], float) == False:
#         rtweetid = df1['referenced_tweets'][index][0]['id']
# #         print(rtweetid)
#         if (df1.loc[df1['id'] == str(rtweetid)]).empty == False:
#             originalrow = df1.loc[df1['id'] == str(rtweetid)]
#             df2['text'][index] = originalrow.iloc[0,0]
# #         print(originalrow)
# #         print(originalrow.iloc[0,0])
# print(df1['referenced_tweets'].values[0])
# def text_iter3(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     if isinstance(ref_tweets, float) == False:
#         rtweetid = ref_tweets
#         if (df1.loc[twid == str(rtweetid)]).empty == False:
#             originalrow = df1.loc[twid == str(rtweetid)]
#             df2['fulltext'] = originalrow.iloc[0,0]

# df2['fulltext'] = text_iter3(df1['referenced_tweets'].values,df1['id'].values)

# df2.head(599999)

# df2['fulltext'] = df1['text']
# df2['fulltext']

# def soc_iter(TEAM,home,away,ftr):
#     #team, row['HomeTeam'], row['AwayTeam'], row['FTR']
#     if [((home == TEAM) & (ftr == 'D')) | ((away == TEAM) & (ftr == 'D'))]:
#         result = 'Draw'
#     elif [((home == TEAM) & (ftr != 'D')) | ((away == TEAM) & (ftr != 'D'))]:
#         result = 'No_Draw'
#     else:
#         result = 'No_Game'
#     return result

# def soc_iter(TEAM,home,away,ftr):
#     df['Draws'] = 'No_Game'
#     df.loc[((home == TEAM) & (ftr == 'D')) | ((away == TEAM) & (ftr == 'D')), 'Draws'] = 'Draw'
#     df.loc[((home == TEAM) & (ftr != 'D')) | ((away == TEAM) & (ftr != 'D')), 'Draws'] = 'No_Draw'

# def text_iter(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     df2.loc[isinstance(ref_tweets, float) == False, df2['rt']] = ref_tweets[0]['id']
#     df2.loc[(df2.loc[twid == str(rtweetid)]).empty == False, df2['fulltext']] = (df2.loc[df2['id'].values == str(rtweetid)]).iloc[0,0]

# def text_iter2(ref_tweets,twid):
#     df2['fulltext'] = df1['text']
#     df2['rt'] = ""
#     df2.loc[isinstance(ref_tweets, float) == False, 'rt'] = ref_tweets
#     df2.loc[twid == str(rtweetid).empty == False, 'fulltext'] = (df2.loc[twid == str(rtweetid)]).iloc[0,0]

    
# df2['fulltext'] = text_iter2(df2['referenced_tweets'].values,df2['id'].values)

[{'type': 'retweeted', 'id': '1391353131334717440'}]


IndexError: list index out of range

In [ ]:
# print(df1['referenced_tweets'][0][0]['id'])
# print(type(df1['id'][0]))

# for index, row in df1.iterrows():
# #     a = row['referenced_tweets']
#     if isinstance(df1['referenced_tweets'][index], float) == False:
#         rtweetid = df1['referenced_tweets'][index][0]['id']
#         originalrow = df1.loc[df1['id'] == rtweetid]
#         df1['text'][index] = originalrow['text']

In [60]:
# df1.to_csv("twitter_aapl_1_clean.csv", index = False)

In [62]:
dffinal2 = dffinal
# Only including this step for now, basically removes the time component, leaving only date
dffinal2['time'] = dffinal2['created_at'].map(lambda x: str(x)[11:])
dffinal2['created_at'] = dffinal2['created_at'].map(lambda x: str(x)[:-9])
dffinal2 = dffinal2.rename(columns={"created_at" : "Date"})
dffinal2

,referenced_tweets,id,text,lang,Date,time
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21
...,...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12,13:00:44
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12,13:00:41
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12,13:00:30
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12,13:00:02


In [34]:
# dffinal2.to_csv("dffinal2.csv", index = False)

dffinal2 = pd.read_csv('dffinal2.csv')

In [63]:
# adapted from https://www.30secondsofcode.org/python/s/is-weekend

def if_weekend(d = datetime.today()):
    if d.weekday() == 5:
        d = d + timedelta(days=2)
    elif d.weekday() == 6:
        d = d + timedelta(days=1)   
        
    if d == date(2021,7,5):
        d = date(2021,7,6)
    elif d == date(2021,5,31):
        d = date(2021,6,1)
    elif d == date(2021,4,2):
        d = date(2021,4,5)
    elif d == date(2021,2,14):
        d = date(2021,2,16)
    elif d == date(2021,2,13):
        d = date(2021,2,16)    
    elif d == date(2021,2,15):#
        d = date(2021,2,16)
    elif d == date(2021,1,16):#
        d = date(2021,1,19) 
    elif d == date(2021,1,17):#
        d = date(2021,1,19)    
    elif d == date(2021,1,18):#
        d = date(2021,1,19)
    elif d == date(2021,1,1):
        d = date(2021,1,4)
    elif d == date(2020,12,25):
        d = date(2020,12,28)
    elif d == date(2020,11,26):
        d = date(2020,11,27)
    elif d == date(2020,9,5):#
        d = date(2020,9,8) 
    elif d == date(2020,9,6):#
        d = date(2020,9,8)    
    elif d == date(2020,9,7):#
        d = date(2020,9,8)
    elif d == date(2020,7,3):
        d = date(2020,7,6)
    elif d == date(2020,5,23):#
        d = date(2020,5,26)
    elif d == date(2020,5,24):#
        d = date(2020,5,26)
    elif d == date(2020,5,25):#
        d = date(2020,5,26)

    return d

def if_weekendmore(daysahead, d = datetime.today()):
    d = if_weekend(d)
    for day in range(daysahead):
        d = d + timedelta(days=1)
        if d.weekday() == 5:
            d = d + timedelta(days=2)
        elif d.weekday() == 6:
            d = d + timedelta(days=1)
        if d == date(2021,7,5):
            d = date(2021,7,6)
        elif d == date(2021,5,31):
            d = date(2021,6,1)
        elif d == date(2021,4,2):
            d = date(2021,4,5)
        elif d == date(2021,2,14):
            d = date(2021,2,16)
        elif d == date(2021,2,13):
            d = date(2021,2,16)    
        elif d == date(2021,2,15):#
            d = date(2021,2,16)
        elif d == date(2021,1,16):#
            d = date(2021,1,19) 
        elif d == date(2021,1,17):#
            d = date(2021,1,19)    
        elif d == date(2021,1,18):#
            d = date(2021,1,19)
        elif d == date(2021,1,1):
            d = date(2021,1,4)
        elif d == date(2020,12,25):
            d = date(2020,12,28)
        elif d == date(2020,11,26):
            d = date(2020,11,27)
        elif d == date(2020,9,5):#
            d = date(2020,9,8) 
        elif d == date(2020,9,6):#
            d = date(2020,9,8)    
        elif d == date(2020,9,7):#
            d = date(2020,9,8)
        elif d == date(2020,7,3):
            d = date(2020,7,6)
        elif d == date(2020,5,23):#
            d = date(2020,5,26)
        elif d == date(2020,5,24):#
            d = date(2020,5,26)
        elif d == date(2020,5,25):#
            d = date(2020,5,26)
    return d
    
print(if_weekend(date(2021,7,4)))
print(if_weekendmore(daysahead = 1, d = date(2021,7,2)))



print(type(date(2021,6,14)))

2021-07-06
2021-07-06
<class 'datetime.date'>


In [64]:
dffinal2['Date']= pd.to_datetime(dffinal2['Date'])
dffinal2['usedate'] = dffinal2['Date'].map(lambda x: if_weekend(x))
dffinal2['usedate1'] = dffinal2['Date'].map(lambda x: if_weekendmore(1,x))
dffinal2['usedate2'] = dffinal2['Date'].map(lambda x: if_weekendmore(2,x))
dffinal2['usedate3'] = dffinal2['Date'].map(lambda x: if_weekendmore(3,x))
dffinal2['usedate4'] = dffinal2['Date'].map(lambda x: if_weekendmore(4,x))
dffinal2['usedate5'] = dffinal2['Date'].map(lambda x: if_weekendmore(5,x))
dffinal2

,referenced_tweets,id,text,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19
...,...,...,...,...,...,...,...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12,13:00:44,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12,13:00:41,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12,13:00:30,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12,13:00:02,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19


In [65]:
dffinal3 = dffinal2
# dffinal3.to_csv("dffinal3.csv", index = False)

In [37]:
dffinal3 = pd.read_csv('dffinal3.csv')
dffinal3

,text,referenced_tweets,id,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5
0,.\nDomain Name for sale\n\nhttps://t.co/f056Wp...,1.391353e+18,1392083055460851716,en,2021-05-11,11:43:58,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
1,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083046459969541,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
2,.\nDomains For sale\n\nhttps://t.co/0YeRs3JJLb...,1.391353e+18,1392083045058940935,en,2021-05-11,11:43:56,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
3,.\nDomain NameFor Sale\n\nhttps://t.co/a2N3rgJ...,1.391354e+18,1392083027929358336,en,2021-05-11,11:43:52,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
4,Savanna Drinkers 🤦\n\n#R500 #RHOA #AceMagashul...,1.392030e+18,1392082985189511168,en,2021-05-11,11:43:41,2021-05-11,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18
...,...,...,...,...,...,...,...,...,...,...,...,...
3390563,Silicone Case Cover Shockproof Holder For Appl...,NaN,1259830641736273921,en,2020-05-11,13:00:06,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390564,#Apple has shown that #iPhone sales fell more ...,NaN,1259830629723766785,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390565,Check it OUT Shadowland by C.M.Gray Read the F...,NaN,1259830628721229826,en,2020-05-11,13:00:03,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18
3390566,Le casque circum-aural d’Apple serait baptisé ...,NaN,1259830627270119427,fr,2020-05-11,13:00:02,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-18


In [66]:
price = pd.read_csv("aapl_holdout.csv")
price['Date'] = price['Date'].map(lambda x: parser.parse(x).isoformat())
# Only including this step for now, basically removes the time component, leaving only date
price['Date'] = price['Date'].map(lambda x: str(x)[:-9])
price['Date']= pd.to_datetime(price['Date'])
# Change me if looking to change examination scope
price = price.rename(columns={"Date" : "usedate"})
price

,usedate,Close/Last,Volume,Open,High,Low,Change
0,2021-07-09,145.11,99890800,142.750,145.650,142.6522,1.87
1,2021-07-08,143.24,105575500,141.580,144.060,140.6650,-1.33
2,2021-07-07,144.57,104911600,143.535,144.890,142.6600,2.55
3,2021-07-06,142.02,108181800,140.070,143.150,140.0700,2.06
4,2021-07-02,139.96,78945570,137.900,140.000,137.7450,2.69
5,2021-07-01,137.27,52485780,136.600,137.330,135.7600,0.31
6,2021-06-30,136.96,63261390,136.170,137.410,135.8700,0.63
7,2021-06-29,136.33,64556080,134.800,136.490,134.3500,1.55
8,2021-06-28,134.78,62111300,133.410,135.245,133.3500,1.67
9,2021-06-25,133.11,70783750,133.460,133.890,132.8100,-0.30


In [67]:
merged = pd.merge(dffinal2, price, how='left', on='usedate')
merged

,referenced_tweets,id,text,lang,Date,time,usedate,usedate1,usedate2,usedate3,usedate4,usedate5,Close/Last,Volume,Open,High,Low,Change
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21,2021-07-12,2021-07-13,2021-07-14,2021-07-15,2021-07-16,2021-07-19,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12,13:00:44,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19,122.77,112172300.0,123.4,124.64,122.25,122.77
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12,13:00:41,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19,122.77,112172300.0,123.4,124.64,122.25,122.77
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12,13:00:30,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19,122.77,112172300.0,123.4,124.64,122.25,122.77
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12,13:00:02,2021-05-12,2021-05-13,2021-05-14,2021-05-17,2021-05-18,2021-05-19,122.77,112172300.0,123.4,124.64,122.25,122.77


In [68]:
pricesmall = price[['usedate','Change']]

pricesmall = pricesmall.rename(columns={"usedate" : "usedate1", "Change" : "Change1"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate1')
pricesmall = pricesmall.rename(columns={"usedate1" : "usedate2", "Change1" : "Change2"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate2')
pricesmall = pricesmall.rename(columns={"usedate2" : "usedate3", "Change2" : "Change3"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate3')
pricesmall = pricesmall.rename(columns={"usedate3" : "usedate4", "Change3" : "Change4"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate4')
pricesmall = pricesmall.rename(columns={"usedate4" : "usedate5", "Change4" : "Change5"})
merged = pd.merge(merged, pricesmall, how='left', on='usedate5')
merged
# def datematch(x,y,z):
#     for index in range(len(merged)):
#         if y[index] == x[index]:
#             return z[index]

# merged['Change1'] = price['Change'].loc[price['usedate'] == merged['usedate1']]
# merged

,referenced_tweets,id,text,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12,13:00:44,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12,13:00:41,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12,13:00:30,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12,13:00:02,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16


In [69]:
print(len(merged['usedate'].unique()))

dfwhatisgoingon = merged
dfwhatisgoingon = dfwhatisgoingon[dfwhatisgoingon['Volume'].isna()]
print(dfwhatisgoingon['usedate'].unique())
print(dfwhatisgoingon['Date'].unique())
dfwhatisgoingon

42
['2021-07-12T00:00:00.000000000']
['2021-07-11T00:00:00.000000000' '2021-07-10T00:00:00.000000000']


,referenced_tweets,id,text,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254759,NaN,1413649314971258880,How much does it cost to manufacture a smartph...,en,2021-07-10,00:00:35,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254760,NaN,1413649276648017920,"#EURUSD All Targets hit, 75 Pips Profit ❤️😍⭐️\...",en,2021-07-10,00:00:26,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254761,NaN,1413649202845036559,"📣 Nuevo Podcast! ""Why Capitalism is the Only M...",en,2021-07-10,00:00:08,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254762,NaN,1413649177192669186,Check it OUT Classics: Why we should encourage...,en,2021-07-10,00:00:02,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
subset_df = merged[merged["usedate"] == '2020-05-11']
column_count = subset_df.count()
print(column_count)

text                 2692
referenced_tweets     946
id                   2692
lang                 2692
Date                 2692
time                 2692
usedate              2692
usedate1             2692
usedate2             2692
usedate3             2692
usedate4             2692
usedate5             2692
Close/Last              0
Volume                  0
Open                    0
High                    0
Low                     0
Change                  0
Change1              2692
Change2              2692
Change3              2692
Change4              2692
Change5              2692
dtype: int64


In [49]:
merged.isna().sum()

text                       0
referenced_tweets    1356882
id                         0
lang                       0
Date                       0
time                       0
usedate                    0
usedate1                   0
usedate2                   0
usedate3                   0
usedate4                   0
usedate5                   0
Close/Last              2692
Volume                  2692
Open                    2692
High                    2692
Low                     2692
Change                  2692
Change1                 3142
Change2                23221
Change3                29977
Change4                38182
Change5                45652
dtype: int64

In [70]:

merged = merged[merged['usedate'] != '2020-05-12']
merged

,referenced_tweets,id,text,lang,Date,time,usedate,usedate1,usedate2,usedate3,...,Volume,Open,High,Low,Change,Change1,Change2,Change3,Change4,Change5
0,1414164954375094276,1414175172081373189,Même Apple a eu du mal à croire en l’impressio...,fr,2021-07-11,10:50:09,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1414173981448495104,1414175103844188162,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:53,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1414071661767372805,1414175068737867777,-@B1 makeover and #BullishOnEOS\n-U.S. legal #...,en,2021-07-11,10:49:45,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1414173981448495104,1414174938345480192,Time for ARCADEFEST! HTML5 games right in your...,en,2021-07-11,10:49:14,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1414143908469043204,1414174716013674500,💡iPadを使いこなそう💡\n\niPadと一緒に買うべきアイテムと\nあったら便利なアイテ...,ja,2021-07-11,10:48:21,2021-07-12,2021-07-13,2021-07-14,2021-07-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448141,NaN,1392464762223566856,"Apples are high in fiber, vitamin C, and vario...",en,2021-05-12,13:00:44,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448142,NaN,1392464747983953923,https://t.co/NgeeWqNxWD - Top 100 Christian So...,en,2021-05-12,13:00:41,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448143,NaN,1392464700999409667,Meet iPadOS 15 Concept.\n#iPadOS15 #iPadOS15Co...,es,2021-05-12,13:00:30,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16
448144,NaN,1392464587048554501,Check it OUT Foretold by P.S. Winn Read the Fi...,en,2021-05-12,13:00:02,2021-05-12,2021-05-13,2021-05-14,2021-05-17,...,112172300.0,123.4,124.64,122.25,122.77,2.2,2.48,-1.18,-1.42,-0.16


In [71]:
merged.to_csv("mergedfullholdout.csv", index = False)